<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# Trabalhando com dados mapeados
Metas de aprendizagem:

- Use diferentes formas de indexação para selecionar dados com base na posição e coordenadas;
- Selecione os intervalos de data e hora;
- Interpolar dados para novas coordenadas.

## Dimensões nomeadas

Conforme mencionado na sessão anterior, as dimensões rotuladas realmente ajudam a tornar o
código menos difícil de entender. Compare a indexação `numpy` pura:


In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

np.random.seed(0)

In [ ]:
# axis0: x, axis1: y
np_array = np.random.randn(3, 4)
np_array[1, 3]

e fatiar (*slicing*):


In [ ]:
np_array[:2, 1:]

com indexação baseada em rótulo:


In [ ]:
arr = xr.DataArray(np_array, dims=("x", "y"))
arr.isel(x=1, y=3)

Ou o equivalente:


In [ ]:
arr[{"x": 1, "y": 3}]

Devido à sintaxe da linguagem, as fatias devem ser construídas manualmente:


In [ ]:
ds = xr.Dataset(
    {
        "a": (("x", "y"), np.random.randn(3, 4)),
        "b": (("x", "y"), np.random.randn(3, 4)),
    }
)
ds.isel(x=slice(None, 2), y=slice(1, None))

Também podemos usar esses nomes para espiar os dados se a visualização automática não for suficiente:


In [ ]:
ds.head(x=2, y=3)

veja também `tail` e` thin`.


## Coordenadas mapeadas e indexação baseada no mapeamento


objetos xarray se tornam muito mais interessantes ao adicionar mapeamento de coordenadas:


In [ ]:
arr = xr.DataArray(
    np.random.randn(4, 6),
    dims=("x", "y"),
    coords={
        "x": [-3.2, 2.1, 5.3, 6.5],
        "y": pd.date_range("2009-01-05", periods=6, freq="M"),
    },
)
arr

Para selecionar dados por mapeamento de coordenadas em vez de índices inteiros, podemos usar a
mesma sintaxe, usando `sel` em vez de `isel`:


In [ ]:
arr.sel(x=5.3, y="2009-04-30")  # ou a.loc[{"x": 5.3, "y": "2009-04-30"}]

isso exigirá que especifiquemos valores exatos. Se não os tivermos, podemos usar
o parâmetro `method` (veja `Dataset.sel` para documentação):


In [ ]:
arr.sel(x=4, y="2009-04-01", method="nearest")

Também podemos selecionar vários valores:


In [ ]:
arr.sel(x=[-3.2, 6.5], y=slice("2009-02-28", "2009-05-31"))

Se, em vez de selecionar dados, quisermos descartá-los, podemos usar `drop_sel`:


In [ ]:
arr.drop_sel(x=[-3.2, 5.3])

### Exercícios


In [ ]:
ds = xr.tutorial.open_dataset("air_temperature")
ds

1. Selecione as primeiras 10 entradas de latitude e 20 a 40 entradas de longitude


In [ ]:
# Seu código aqui

2. Selecione todos os dados a 75 graus norte e entre 1º de janeiro de 2013 e 15 de outubro de 2013


In [ ]:
# Seu código aqui

3. Remova todas as entradas em 260 e 270 graus


In [ ]:
# Seu código aqui

## Interpolação

Se quisermos ver os valores entre as células da malha atual (interpolação),
podemos fazer isso com `interp` (requer `scipy`):


In [ ]:
arr.interp(
    x=np.linspace(2, 6, 11, endpoint=True),
    y=pd.date_range("2009-04-01", "2009-04-30", freq="D"),
)

Ao tentar extrapolar, os valores resultantes serão `nan`.

Se já temos um objeto com as coordenadas desejadas, podemos usar
`Interp_like`:


In [ ]:
other = xr.DataArray(
    dims=("x", "y"),
    coords={
        "x": np.linspace(2, 4, 10),
        "y": pd.date_range("2009-04-01", "2009-04-30", freq="D"),
    },
)
arr.interp_like(other)

### Exercícios

Aumente o tamanho do passo ao longo da latitude e longitude de 2,5 graus para 1
grau:


In [ ]:
# Seu código aqui

## Propagação (*Broadcasting*) e alinhamento automático

Os mapeamentos ajudam a combinar matrizes com diferentes coordenadas:


In [ ]:
a = xr.DataArray(
    np.random.randn(3, 4),
    dims=("x", "y"),
    coords={"x": ["a", "b", "c"], "y": np.arange(4)},
)
b = xr.DataArray(
    np.random.randn(2, 7),
    dims=("x", "y"),
    coords={"x": ["b", "d"], "y": [-2, -1, 0, 1, 2, 3, 4]},
)

a + b

Isso irá selecionar automaticamente apenas rótulos comuns de ambas as matrizes, e, em seguida, execute a operação.


A propagação (*broadcasting*) funciona de maneira semelhante:


In [ ]:
arr1 = xr.DataArray(
    np.random.randn(3),
    dims="x",
    coords={"x": ["a", "b", "c"]},
)
arr2 = xr.DataArray(
    np.random.randn(4),
    dims="y",
    coords={"y": np.arange(4)},
)

arr1 + arr2

onde ambas as matrizes foram propagadas automaticamente uma contra a outra:


In [ ]:
arr1_, arr2_ = xr.broadcast(arr1, arr2)

In [ ]:
arr1_

In [ ]:
arr2_

e então a operação (uma soma) foi executada.

Também podemos chamar `align` especificamente com opções diferentes.


In [ ]:
a_al, b_al = xr.align(a, b, join="inner")
b_al